In [34]:
from bs4 import BeautifulSoup
import pathlib
import requests as req
import pandas as pd

In [4]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [5]:
resp = req.get(url)

In [8]:
soup = BeautifulSoup(resp.text)

In [33]:
# soup.head()

In [14]:
lineups_html = soup.find('div', attrs={'class': 'lineups'})

In [39]:
type(lineups_html)

bs4.element.Tag

In [36]:
raw_filename = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_lineups/2021/03/20/rotowire.html'

In [40]:
with open(raw_filename, 'w', encoding='utf-8') as fp:
    fp.write(str(lineups_html))

In [41]:
with open(raw_filename, 'r') as fp:
    soup = BeautifulSoup(fp.read())

In [45]:
soup.find_all()[0].find_all('div', attrs={'class': 'lineup__teams'})

1

In [ ]:
lineups_html

In [91]:
matchups_html = [
    x for x 
    in lineups_html.find_all('div', attrs={'class': 'lineup__box'})
    if 'lineup__teams' in str(x)
    ]

In [117]:
lineup_players_list = []
for matchup_html in matchups_html:
    matchup_teams_html = matchup_html.find('div', attrs={'class': 'lineup__teams'})
    matchup = ''.join([x.text for x in matchup_teams_html.find_all('div')])
    matchup_teams = [
        {
            'TEAM_ABBREVIATION': x.find('div').text,
            'HOME_AWAY': x.get('class')[-1],
            'MATCHUP': matchup,
        }
        for x in matchup_teams_html.find_all('a')
    ]
    for team in matchup_teams:
        abbrev = team.get('TEAM_ABBREVIATION')
        home_away = team.get('HOME_AWAY')
        lineup_raw = [x for x in matchup_html.find_all('ul') if home_away in str(x)][0]
        players_raw = [x for x in lineup_raw.find_all('li') if 'lineup__player' in str(x)]
        for player in players_raw:
            player_dict = {
                'TEAM_ABBREVIATION': abbrev,
                'HOME_AWAY': home_away,
                'MATCHUP': matchup,
                'TO_PLAY_DESC': player.get('title'),
                'PLAYER_NAME': player.find('a').get('title'),
                'NAME_SHORT': player.find('a').text,
                'STARTING_POSITION': player.find(
                    'div', attrs={'class', 'lineup__pos'}).text,
            }
            lineup_players_list.append(player_dict)

In [114]:
lineup_df = pd.DataFrame(lineup_players_list).drop_duplicates()

In [115]:
lineup_df.to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_lineups/2021/03/20/rotowire.csv', index=False)